In [ ]:
# customer_segmentation.ipynb

# Importing the necessary libraries for data analysis and visualization
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
import plotly.express as px

In [ ]:
# Load the data
# The customer data CSV file is loaded here
data = pd.read_csv('data/customer_data.csv')

# Show the first few rows of the dataset to understand its structure
data.head()

